# Compare boms
* This page is for check the result of material classify
* The way check is compare the mateiral ID in the result material_list and the original bom with ID added by funcion *B_with_ID()*.
* Then generator another csv with on the difference material between tow data, by funciton *remain_diff()*

In [455]:
import pandas as pd
import numpy as np
# File system
from os import walk
from os.path import join

#Switch area
SHEET_NAME_LIST = ['bom', 'trims', 'shell', 'fabric', 'accessories', 'spec', '228184', '#334183', 'tabelle1']
## Some column of bom have unparsible value than will affect the function df.drop_duplicates(). 
## If sutiation above happens, turn the variable to be True, to filter only specific columns only
SPECIAL_FILTER = True

#Path
BOM ='DataSource/Original_BOM'
CSV_1_DIR ='result/CSV_1_DIR'
CSV_2_DIR = 'DataSource/Material_list'
RESULT = 'result'

xl = pd.ExcelFile(TEST)

## Convert bom to csv with ID
* Comvert a bom into a csv with a column "MATERIAL_ID" with index numbers

In [456]:
def B_with_ID(filepath, TO_DIR, file_name):
    xl = pd.ExcelFile(filepath)
    for sheet in xl.sheet_names:
        name = str(sheet).lower()
        if any(x in name for x in SHEET_NAME_LIST): 
            bom = xl.parse(sheet, index_col = None, header = None)
            
            bom.index.name = 'MATERIAL_ID'
#             Delete empty columns
            for col in bom:
                if bom[col].count() == 0:
                    bom = bom.drop(col, axis = 1)
                    
            bom.to_csv(TO_DIR + '/' + file_name + '_'+ sheet + '.csv')  
    return

* Walk through a dir

In [457]:
def B_with_ID_walk_DIR(FROM_DIR, TO_DIR):
    converted_xlsm_num = 0
    for root, dirnames, filenames in walk(FROM_DIR):
        # walk through each xlsx file
        for file_name in filenames:
             # get the path of the file
            # Appoint the method only work with .xlsx file.
            if file_name.endswith('.xlsx') :
                converted_xlsm_num = converted_xlsm_num + 1
                filepath = join(root, file_name)
                # Custom function
                B_with_ID(filepath, TO_DIR, file_name[0:-5])
    print('Converted ', converted_xlsm_num, ' files')

In [458]:
%%time
B_with_ID_walk_DIR(BOM, CSV_1_DIR)

Converted  5  files
CPU times: user 74 ms, sys: 4.98 ms, total: 79 ms
Wall time: 79.2 ms


## Compare two csv and remain the difference

In [459]:
def remain_diff(CSV_1, CSV_2, filename, TO_DIR, SPECIAL_FILTER):
    csv_1 = pd.read_csv(CSV_1, index_col = 0)
    csv_2 = pd.read_csv(CSV_2 + '/' + filename + '_material_list.csv', index_col = 0)
    if SPECIAL_FILTER == False:
        diff_df = pd.concat([csv_1, csv_2], sort = False).drop_duplicates(keep = False)
    else:
        col_to_check = []
        # Decide how many col to check.
        for num in range(0, 3):
            col_to_check.append(csv_2.columns[num])
        diff_df = pd.concat([csv_1, csv_2], sort = False).drop_duplicates(keep = False, subset = col_to_check)
        
    diff_df.to_csv(TO_DIR + '/_diff_' + filename + '.csv' )

* Walk through a dir

In [460]:
def remain_diff_walk_DIR(FROM_DIR_1, FROM_DIR_2, TO_DIR, SPECIAL_FILTER):
    converted_xlsm_num = 0
    for root, dirnames, filenames in walk(FROM_DIR_1):
        # walk through each xlsx file
        for file_name in filenames:
             # get the path of the file
            # Appoint the method only work with .xlsx file.
            if file_name.endswith('.csv') :
                converted_xlsm_num = converted_xlsm_num + 1
                CSV_1_filepath = join(root, file_name)
                # Custom function
                remain_diff(CSV_1_filepath, FROM_DIR_2, file_name[0:-4], TO_DIR, SPECIAL_FILTER )
                    
    print('Converted ', converted_xlsm_num, ' files')

In [461]:
%%time
remain_diff_walk_DIR(CSV_1_DIR, CSV_2_DIR, RESULT, SPECIAL_FILTER)

Converted  5  files
CPU times: user 50.4 ms, sys: 4.96 ms, total: 55.4 ms
Wall time: 55 ms
